In [3]:
pip install selenium

^C
Note: you may need to restart the kernel to use updated packages.


ERROR: After October 2020 you may experience errors when installing or updating packages. This is because pip will change the way that it resolves dependency conflicts.

We recommend you use --use-feature=2020-resolver to test your packages with the new resolver before it becomes the default.

requests 2.24.0 requires urllib3!=1.25.0,!=1.25.1,<1.26,>=1.21.1, but you'll have urllib3 1.26.12 which is incompatible.


  Attempting uninstall: urllib3
    Found existing installation: urllib3 1.25.11
    Uninstalling urllib3-1.25.11:
      Successfully uninstalled urllib3-1.25.11
  Attempting uninstall: certifi
    Found existing installation: certifi 2020.6.20
    Uninstalling certifi-2020.6.20:
      Successfully uninstalled certifi-2020.6.20


In [4]:
from bs4 import BeautifulSoup
from selenium import webdriver
from selenium.webdriver.common.by import By
from selenium.webdriver.common.keys import Keys
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC
import time
import numpy as np
import pickle

In [64]:
#log into linked in
def login(driver,p):
  url =  "https://www.linkedin.com/checkpoint/rm/sign-in-another-account?fromSignIn=true&trk=guest_homepage-basic_nav-header-signin"
  wait = WebDriverWait(driver, 10)
  driver.get(url)
  
  username = driver.find_element('id', "username")
  username.send_keys("cae.aragao.simoes@gmail.com")
  password = driver.find_element('id', "password")
  password.send_keys(p)
  driver.find_element(By.CLASS_NAME,"login__form_action_container").click()


#redirects driver to the query link
def search(driver):
  time.sleep(5)
  driver.get("https://www.linkedin.com/jobs/search/?keywords=data%20science")
    

#grabs results fetched
def get_n_results(driver):
  time.sleep(10)
  results_div = driver.find_element(By.XPATH, "/html/body/div[5]/div[3]/div[3]/div[2]/div/section[1]/div/header/div[1]/small")
  n_string = results_div.text
  n = int(n_string.split()[0].replace(',',""))
  return n 


#Finds job ul div
def get_jobs(driver):
  ul_div = driver.find_element(By.XPATH, '//*[@id="main"]/div/section[1]/div')
  return ul_div


#Scrolls to properly load page
def scroll_down(driver):
  for i in np.linspace(0,1,10):
    time.sleep(2)
    driver.execute_script(f"window.scrollTo(0,document.body.scrollHeight*{i})")
    

def get_job_urls(jobs,driver,job_urls = {}):
  i = 1
  #Collects job urls,location role cand company 
  #the final result updates the input dictionary and appends a key value pair with the format
  #    url:{'company':company,'location':location,'role':role}
  while True: 
    try:
      WebDriverWait(driver, 1).until(EC.presence_of_element_located((By.XPATH,f'/html/body/div[5]/div[3]/div[3]/div[2]/div/section[1]/div/div/ul/li[{i}]')))
      url = jobs.find_element(By.XPATH, f'/html/body/div[5]/div[3]/div[3]/div[2]/div/section[1]/div/div/ul/li[{i}]/div/div/div[1]/div[2]/div[1]/a').get_attribute("href")
      role = jobs.find_element(By.XPATH, f'/html/body/div[5]/div[3]/div[3]/div[2]/div/section[1]/div/div/ul/li[{i}]/div/div/div[1]/div[2]/div[1]/a').text
      company = jobs.find_element(By.XPATH, f'/html/body/div[5]/div[3]/div[3]/div[2]/div/section[1]/div/div/ul/li[{i}]/div/div/div[1]/div[2]/div[1]/a').text
      location = driver.find_element(By.XPATH, f'/html/body/div[5]/div[3]/div[3]/div[2]/div/section[1]/div/div/ul/li[{i}]/div/div/div[1]/div[2]/div[3]/ul/li').text
      job_urls.update({url:{'company':company,'location':location,'role':role}})
      i+=1
    except:
      return job_urls


def load_next_page(driver):
  #loads next page for url retrival
  curr= driver.find_element(By.XPATH, '//*[@aria-current="true"]').text
  next = driver.find_element(By.XPATH, f'//*[@aria-label="Página {int(curr)+1}"]')
  next.click()


def get_description(driver,job_dict,good):

  fail = []
  #Iterate through the url list to scrape the descriptions
  for url in list(job_dict.keys()):
    if url not in good:
      try:
        driver.get(url)
        time.sleep(3)
        if driver.current_url != url:
          print(f'failed at {url}')
          #remove borken urls
          job_dict.pop(url)
        #scrape
        driver.find_element(By.XPATH, '//*[@aria-label="Click to see more description"]').click()
        description = driver.find_element(By.XPATH, '/html/body/div[6]/div[3]/div/div[1]/div[1]/div/div[2]/article/div').text
        job_dict.get(url).update({"description":description})
        good.append(url)
        return job_dict
      except:
        #keep going if there is a random error in which a div did not load properly but check where we failed
        print(f"fail {job_dict.get(url)}")
        fail.append(url)


In [65]:
#Run theough the entire process of fetch in urls logining in and grabing job descriptions
def main(driver,password):
    login(driver,password)
    search(driver)

    job_dict ={}
    #iterate through the amount of pages given
    for i in range(39):
        jobs = get_jobs(driver)
        scroll_down(driver)
        get_job_urls(jobs,driver,job_urls = job_dict)
        load_next_page(driver)
    get_description(driver,job_dict,good)

    return get_description(driver,job_dict,good)

In [63]:
driver = webdriver.Chrome(r'C:\Program Files (x86)\Google\chromedriver.exe')
main(driver, 'GreatTie')

<ipython-input-63-92116c521bc9>:1: DeprecationWarning: executable_path has been deprecated, please pass in a Service object
  driver = webdriver.Chrome(r'C:\Program Files (x86)\Google\chromedriver.exe')


ElementClickInterceptedException: Message: element click intercepted: Element <button aria-label="Página 16" type="button" data-ember-action="" data-ember-action-2173="2173">...</button> is not clickable at point (328, 823). Other element would receive the click: <section class="scaffold-layout__list" tabindex="-1">...</section>
  (Session info: chrome=106.0.5249.119)
Stacktrace:
Backtrace:
	Ordinal0 [0x00AC1ED3+2236115]
	Ordinal0 [0x00A592F1+1807089]
	Ordinal0 [0x009666FD+812797]
	Ordinal0 [0x0099BEC7+1031879]
	Ordinal0 [0x00999E6C+1023596]
	Ordinal0 [0x00997A5B+1014363]
	Ordinal0 [0x009966E7+1009383]
	Ordinal0 [0x0098C416+967702]
	Ordinal0 [0x009B1A8C+1120908]
	Ordinal0 [0x0098BD84+966020]
	Ordinal0 [0x009B1CA4+1121444]
	Ordinal0 [0x009C59E2+1202658]
	Ordinal0 [0x009B18A6+1120422]
	Ordinal0 [0x0098A73D+960317]
	Ordinal0 [0x0098B71F+964383]
	GetHandleVerifier [0x00D6E7E2+2743074]
	GetHandleVerifier [0x00D608D4+2685972]
	GetHandleVerifier [0x00B52BAA+532202]
	GetHandleVerifier [0x00B51

In [66]:
#use pickle object storage to store dictionary items for later use

def save(final_file):
    with open(f'../data/{final_file}.p', 'wb') as fp:
        pickle.dump(final_file,file=fp,protocol=pickle.HIGHEST_PROTOCOL)
def load():
    with open('../data/job_dict.p', 'rb') as fp:
        job_dict = pickle.load(fp)